In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("VarunSparkWindows").master("local[5]").getOrCreate()

25/06/12 06:05:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/12 06:05:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/12 06:05:10 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/12 06:05:10 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/06/12 06:05:10 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/06/12 06:05:10 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/06/12 06:05:10 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/06/12 06:05:10 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/06/12 06:05:10 WARN Utils: Serv

In [3]:
spark

In [6]:
from pyspark.sql.types import *

In [7]:
sch = StructType([StructField("name", StringType(),True),StructField("department",StringType(),True),StructField("salary",IntegerType(),True)])

In [11]:
windowdata = spark.read.csv('pysparkasset/windowdata.txt',sep=',', schema=sch)

In [12]:
windowdata.show()

+-------+----------+------+
|   name|department|salary|
+-------+----------+------+
|  James|     Sales|  3000|
|Michael|     Sales|  4600|
| Robert|     Sales|  4100|
|  Maria|   Finance|  3000|
|    Ram|     Sales|  3000|
|  Scott|   Finance|  3300|
|    Jen|   Finance|  3900|
|   Jeff| Marketing|  3000|
|  Kumar| Marketing|  2000|
| Sundar|     Sales|  4100|
|   Amit|     Sales|  4600|
+-------+----------+------+



In [13]:
windowdata.printSchema()

root
 |-- name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: integer (nullable = true)



### Window Functions

In [15]:
from pyspark.sql.functions import *

In [16]:
from pyspark.sql.window import *

In [17]:
win1 = Window.partitionBy("department")

In [26]:
resdf = windowdata.withColumn("msal", max('salary').over(win1))

In [27]:
resdf.show()

+-------+----------+------+----+
|   name|department|salary|msal|
+-------+----------+------+----+
|  James|     Sales|  3000|4600|
|Michael|     Sales|  4600|4600|
| Robert|     Sales|  4100|4600|
|    Ram|     Sales|  3000|4600|
| Sundar|     Sales|  4100|4600|
|   Amit|     Sales|  4600|4600|
|  Maria|   Finance|  3000|3900|
|  Scott|   Finance|  3300|3900|
|    Jen|   Finance|  3900|3900|
|   Jeff| Marketing|  3000|3000|
|  Kumar| Marketing|  2000|3000|
+-------+----------+------+----+



In [28]:
resdf1 = resdf.withColumn("diffcol", max('salary').over(win1) - col("salary"))

In [30]:
resdf1.show()

+-------+----------+------+----+-------+
|   name|department|salary|msal|diffcol|
+-------+----------+------+----+-------+
|  James|     Sales|  3000|4600|   1600|
|Michael|     Sales|  4600|4600|      0|
| Robert|     Sales|  4100|4600|    500|
|    Ram|     Sales|  3000|4600|   1600|
| Sundar|     Sales|  4100|4600|    500|
|   Amit|     Sales|  4600|4600|      0|
|  Maria|   Finance|  3000|3900|    900|
|  Scott|   Finance|  3300|3900|    600|
|    Jen|   Finance|  3900|3900|      0|
|   Jeff| Marketing|  3000|3000|      0|
|  Kumar| Marketing|  2000|3000|   1000|
+-------+----------+------+----+-------+



In [31]:
win2 = Window.partitionBy('department').orderBy('salary')

In [33]:
resrow = windowdata.withColumn('rnum',row_number().over(win2))

In [34]:
resrow.show()

+-------+----------+------+----+
|   name|department|salary|rnum|
+-------+----------+------+----+
|  James|     Sales|  3000|   1|
|    Ram|     Sales|  3000|   2|
| Robert|     Sales|  4100|   3|
| Sundar|     Sales|  4100|   4|
|Michael|     Sales|  4600|   5|
|   Amit|     Sales|  4600|   6|
|  Maria|   Finance|  3000|   1|
|  Scott|   Finance|  3300|   2|
|    Jen|   Finance|  3900|   3|
|  Kumar| Marketing|  2000|   1|
|   Jeff| Marketing|  3000|   2|
+-------+----------+------+----+



In [35]:
rankrow = windowdata.withColumn('ranknum',rank().over(win2))

In [36]:
rankrow.show()

+-------+----------+------+-------+
|   name|department|salary|ranknum|
+-------+----------+------+-------+
|  James|     Sales|  3000|      1|
|    Ram|     Sales|  3000|      1|
| Robert|     Sales|  4100|      3|
| Sundar|     Sales|  4100|      3|
|Michael|     Sales|  4600|      5|
|   Amit|     Sales|  4600|      5|
|  Maria|   Finance|  3000|      1|
|  Scott|   Finance|  3300|      2|
|    Jen|   Finance|  3900|      3|
|  Kumar| Marketing|  2000|      1|
|   Jeff| Marketing|  3000|      2|
+-------+----------+------+-------+



In [38]:
denserankrow = windowdata.withColumn('ranknum',dense_rank().over(win2))

In [39]:
denserankrow.show()

+-------+----------+------+-------+
|   name|department|salary|ranknum|
+-------+----------+------+-------+
|  James|     Sales|  3000|      1|
|    Ram|     Sales|  3000|      1|
| Robert|     Sales|  4100|      2|
| Sundar|     Sales|  4100|      2|
|Michael|     Sales|  4600|      3|
|   Amit|     Sales|  4600|      3|
|  Maria|   Finance|  3000|      1|
|  Scott|   Finance|  3300|      2|
|    Jen|   Finance|  3900|      3|
|  Kumar| Marketing|  2000|      1|
|   Jeff| Marketing|  3000|      2|
+-------+----------+------+-------+



In [50]:
leadrow = windowdata.withColumn('leadsal',lead("salary").over(win2))

In [52]:
leadrow.show()

+-------+----------+------+-------+
|   name|department|salary|leadsal|
+-------+----------+------+-------+
|  James|     Sales|  3000|   3000|
|    Ram|     Sales|  3000|   4100|
| Robert|     Sales|  4100|   4100|
| Sundar|     Sales|  4100|   4600|
|Michael|     Sales|  4600|   4600|
|   Amit|     Sales|  4600|   null|
|  Maria|   Finance|  3000|   3300|
|  Scott|   Finance|  3300|   3900|
|    Jen|   Finance|  3900|   null|
|  Kumar| Marketing|  2000|   3000|
|   Jeff| Marketing|  3000|   null|
+-------+----------+------+-------+



In [53]:
lagrow = windowdata.withColumn('lagnum',lag("salary").over(win2))

In [54]:
lagrow.show()

+-------+----------+------+------+
|   name|department|salary|lagnum|
+-------+----------+------+------+
|  James|     Sales|  3000|  null|
|    Ram|     Sales|  3000|  3000|
| Robert|     Sales|  4100|  3000|
| Sundar|     Sales|  4100|  4100|
|Michael|     Sales|  4600|  4100|
|   Amit|     Sales|  4600|  4600|
|  Maria|   Finance|  3000|  null|
|  Scott|   Finance|  3300|  3000|
|    Jen|   Finance|  3900|  3300|
|  Kumar| Marketing|  2000|  null|
|   Jeff| Marketing|  3000|  2000|
+-------+----------+------+------+



--------------

In [55]:
purple = spark.read.format('text').load('purplecow.txt')

In [56]:
purple.show()

+--------------------+
|               value|
+--------------------+
|I've never seen a...|
|I never hope to s...|
|But I can tell yo...|
|I'd rather see th...|
+--------------------+



In [63]:
df = purple.select(explode(split("value",' ')).alias('words'))

In [64]:
df.show()

+-------+
|  words|
+-------+
|   I've|
|  never|
|   seen|
|      a|
| purple|
|   cow.|
|      I|
|  never|
|   hope|
|     to|
|    see|
|   one;|
|    But|
|      I|
|    can|
|   tell|
|   you,|
|anyhow,|
|    I'd|
| rather|
+-------+
only showing top 20 rows



---------------

In [66]:
df.groupBy('words').count().show()

+-------+-----+
|  words|count|
+-------+-----+
|   hope|    1|
|    But|    1|
|    I'd|    1|
|   cow.|    1|
|   seen|    1|
|    can|    1|
| rather|    1|
|   tell|    1|
|anyhow,|    1|
|     be|    1|
|   than|    1|
|   one.|    1|
| purple|    1|
|   I've|    1|
|   you,|    1|
|   one;|    1|
|    see|    2|
|  never|    2|
|      I|    2|
|      a|    1|
+-------+-----+
only showing top 20 rows



posexplode()

-----------------

## User-Defined Functions

In [68]:
def touppercases(x):
    return x.upper()

In [69]:
touppercases('india')

'INDIA'

### Steps to write UDF

- create a UDF using python
- Register UDF with spark SQL

In [70]:
myucase = udf(lambda x:touppercases(x),StringType())

In [71]:
windowdata.withColumn('uc',myucase(col('name'))).show()

+-------+----------+------+-------+
|   name|department|salary|     uc|
+-------+----------+------+-------+
|  James|     Sales|  3000|  JAMES|
|Michael|     Sales|  4600|MICHAEL|
| Robert|     Sales|  4100| ROBERT|
|  Maria|   Finance|  3000|  MARIA|
|    Ram|     Sales|  3000|    RAM|
|  Scott|   Finance|  3300|  SCOTT|
|    Jen|   Finance|  3900|    JEN|
|   Jeff| Marketing|  3000|   JEFF|
|  Kumar| Marketing|  2000|  KUMAR|
| Sundar|     Sales|  4100| SUNDAR|
|   Amit|     Sales|  4600|   AMIT|
+-------+----------+------+-------+



In [79]:
windowdata.createOrReplaceTempView('employee')

In [81]:
spark.udf.register("uppcase",touppercases,StringType())

<function __main__.touppercases(x)>

In [82]:
spark.sql("select name,uppcase(name) from employee").show()

+-------+-------------+
|   name|uppcase(name)|
+-------+-------------+
|  James|        JAMES|
|Michael|      MICHAEL|
| Robert|       ROBERT|
|  Maria|        MARIA|
|    Ram|          RAM|
|  Scott|        SCOTT|
|    Jen|          JEN|
|   Jeff|         JEFF|
|  Kumar|        KUMAR|
| Sundar|       SUNDAR|
|   Amit|         AMIT|
+-------+-------------+



In [85]:
ac_par_part = spark.read.csv('account_parquet_csv', sep=',', inferSchema=True)

In [87]:
ac_par_part.createTempView('acc_par_csv')

In [89]:
spark.sql('select * from acc_par_csv').explain()

== Physical Plan ==
FileScan csv [_c0#2335,_c1#2336L,_c2#2337L,_c3#2338,_c4#2339,_c5#2340,_c6#2341,_c7#2342,_c8#2343L,_c9#2344L,_c10#2345L,state#2346] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://master:9000/user/varunm15t38hedu/account_parquet_csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:int,_c1:bigint,_c2:bigint,_c3:string,_c4:string,_c5:string,_c6:string,_c7:int,_c8:bigi...




In [90]:
ac_par_part.where(ac_par_part.state=='CA').explain()

== Physical Plan ==
FileScan csv [_c0#2335,_c1#2336L,_c2#2337L,_c3#2338,_c4#2339,_c5#2340,_c6#2341,_c7#2342,_c8#2343L,_c9#2344L,_c10#2345L,state#2346] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://master:9000/user/varunm15t38hedu/account_parquet_csv], PartitionFilters: [isnotnull(state#2346), (state#2346 = CA)], PushedFilters: [], ReadSchema: struct<_c0:int,_c1:bigint,_c2:bigint,_c3:string,_c4:string,_c5:string,_c6:string,_c7:int,_c8:bigi...




In [102]:
ac_par_part.where(((ac_par_part.state=='CA')&(ac_par_part.state==''))).explain()

== Physical Plan ==
FileScan csv [_c0#2335,_c1#2336L,_c2#2337L,_c3#2338,_c4#2339,_c5#2340,_c6#2341,_c7#2342,_c8#2343L,_c9#2344L,_c10#2345L,state#2346] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://master:9000/user/varunm15t38hedu/account_parquet_csv], PartitionFilters: [isnotnull(state#2346), (state#2346 = CA), (state#2346 = )], PushedFilters: [], ReadSchema: struct<_c0:int,_c1:bigint,_c2:bigint,_c3:string,_c4:string,_c5:string,_c6:string,_c7:int,_c8:bigi...




### Note: `pushed_filters` and `partition_filters` only works on dataframes in which data is not partitioned.

In [107]:
ac_par = spark.read.format('parquet').load('accounts_parquet')

In [116]:
ac_par.where(ac_par.state=='CA').select('state').explain('formatted')

== Physical Plan ==
* Filter (3)
+- * ColumnarToRow (2)
   +- Scan parquet  (1)


(1) Scan parquet 
Output [1]: [state#2699]
Batched: true
Location: InMemoryFileIndex [hdfs://master:9000/user/varunm15t38hedu/accounts_parquet]
PushedFilters: [IsNotNull(state), EqualTo(state,CA)]
ReadSchema: struct<state:string>

(2) ColumnarToRow [codegen id : 1]
Input [1]: [state#2699]

(3) Filter [codegen id : 1]
Input [1]: [state#2699]
Condition : (isnotnull(state#2699) AND (state#2699 = CA))




In [117]:
ac_par.groupBy('state').count().explain('formatted')

== Physical Plan ==
* HashAggregate (5)
+- Exchange (4)
   +- * HashAggregate (3)
      +- * ColumnarToRow (2)
         +- Scan parquet  (1)


(1) Scan parquet 
Output [1]: [state#2699]
Batched: true
Location: InMemoryFileIndex [hdfs://master:9000/user/varunm15t38hedu/accounts_parquet]
ReadSchema: struct<state:string>

(2) ColumnarToRow [codegen id : 1]
Input [1]: [state#2699]

(3) HashAggregate [codegen id : 1]
Input [1]: [state#2699]
Keys [1]: [state#2699]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#2870L]
Results [2]: [state#2699, count#2871L]

(4) Exchange
Input [2]: [state#2699, count#2871L]
Arguments: hashpartitioning(state#2699, 200), ENSURE_REQUIREMENTS, [id=#991]

(5) HashAggregate [codegen id : 2]
Input [2]: [state#2699, count#2871L]
Keys [1]: [state#2699]
Functions [1]: [count(1)]
Aggregate Attributes [1]: [count(1)#2866L]
Results [2]: [state#2699, count(1)#2866L AS count#2867L]




In [111]:
ac_par_part.groupBy('state').count().explain()

== Physical Plan ==
*(2) HashAggregate(keys=[state#2346], functions=[count(1)])
+- Exchange hashpartitioning(state#2346, 200), ENSURE_REQUIREMENTS, [id=#881]
   +- *(1) HashAggregate(keys=[state#2346], functions=[partial_count(1)])
      +- FileScan csv [state#2346] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://master:9000/user/varunm15t38hedu/account_parquet_csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<>




In [113]:
ppl = spark.read.json('pysparkasset/people.json')

In [114]:
ppl.join(ac_par, ppl.pcode==ac_par.zipcode,"inner").explain()

== Physical Plan ==
*(2) BroadcastHashJoin [pcode#2789], [zipcode#2700], Inner, BuildLeft, false
:- BroadcastExchange HashedRelationBroadcastMode(List(input[2, string, false]),false), [id=#916]
:  +- *(1) Filter isnotnull(pcode#2789)
:     +- FileScan json [age#2787L,name#2788,pcode#2789] Batched: false, DataFilters: [isnotnull(pcode#2789)], Format: JSON, Location: InMemoryFileIndex[hdfs://master:9000/user/varunm15t38hedu/pysparkasset/people.json], PartitionFilters: [], PushedFilters: [IsNotNull(pcode)], ReadSchema: struct<age:bigint,name:string,pcode:string>
+- *(2) Filter isnotnull(zipcode#2700)
   +- *(2) ColumnarToRow
      +- FileScan parquet [acct_num#2692,acct_create_dt#2693L,acct_close_dt#2694L,first_name#2695,last_name#2696,address#2697,city#2698,state#2699,zipcode#2700,phone_number#2701,created#2702L,modified#2703L] Batched: true, DataFilters: [isnotnull(zipcode#2700)], Format: Parquet, Location: InMemoryFileIndex[hdfs://master:9000/user/varunm15t38hedu/accounts_parquet], Par

In [115]:
ppl.join(ac_par, ppl.pcode==ac_par.zipcode,"inner").explain('formatted')

== Physical Plan ==
* BroadcastHashJoin Inner BuildLeft (7)
:- BroadcastExchange (3)
:  +- * Filter (2)
:     +- Scan json  (1)
+- * Filter (6)
   +- * ColumnarToRow (5)
      +- Scan parquet  (4)


(1) Scan json 
Output [3]: [age#2787L, name#2788, pcode#2789]
Batched: false
Location: InMemoryFileIndex [hdfs://master:9000/user/varunm15t38hedu/pysparkasset/people.json]
PushedFilters: [IsNotNull(pcode)]
ReadSchema: struct<age:bigint,name:string,pcode:string>

(2) Filter [codegen id : 1]
Input [3]: [age#2787L, name#2788, pcode#2789]
Condition : isnotnull(pcode#2789)

(3) BroadcastExchange
Input [3]: [age#2787L, name#2788, pcode#2789]
Arguments: HashedRelationBroadcastMode(List(input[2, string, false]),false), [id=#951]

(4) Scan parquet 
Output [12]: [acct_num#2692, acct_create_dt#2693L, acct_close_dt#2694L, first_name#2695, last_name#2696, address#2697, city#2698, state#2699, zipcode#2700, phone_number#2701, created#2702L, modified#2703L]
Batched: true
Location: InMemoryFileIndex [hdfs:/